# Furniture Company


**This is the inital version of the linear model for the furniture problem and serves as a base for the other models in this project**


We decided to use a three part exercise consisting of a production problem, a workforce problem and a transportation problem. The three parts can be solved seperatly one after the other, were problems use data from prior parts or at the same time as a big more complex problem.

## Seperate Problems

First we define each problem seperately with parameters to give on to the next problem.

### Production

The company has two branches.
Branch one:
* produces chairs and desks
* Each chair requires 
    * 1 unit of wood, 1 unit of metal
    * 2 hours in workshop A, 1 hour in workshop B
    * brings a net profit of 3€
* Each desk requires
    * 2 units of wood, 1 unit of metal
    * 3 hours in workshop A, 1 hour in workshop B
    * brings a net profit of 4€
* 120 hours are available in workshop A
* 50 hours are available in workshop B

Branch two:
* produces bedframes and bookcases
* each bedframe requires
    * 2 units of wood, 1 unit of metal
    * 1 hour in workshop C, 2 hours in workshop D
    * brings a net profit of 4€
* each bookcase requires
    * 3 units of wood, 1 unit of metal
    * 3 hours in workshop C, 1 hour in workshop D
    * brings a net profit of 7€
* 150 hours are available in workshop C
* 100 hours are available in workshop D
    
Overal there are only 225 units of wood and 114 units of metal available

We can simulate this problem for several months by adding fluctuations to the net profits of each item.

#### Fluctuations:

|     | J | F | M | A | M | J | J | A | S | O | N | D |
|-----|---|---|---|---|---|---|---|---|---|---|---|---|
|Chair| 3 | 3 | 4 | 3 | 2 | 2 | 3 | 3 | 4 | 2 | 3 | 4 |
|Desk | 5 | 3 | 4 | 3 | 5 | 4 | 3 | 5 | 4 | 4 | 3 | 4 |
|Bed  | 3 | 5 | 4 | 3 | 3 | 3 | 3 | 5 | 4 | 4 | 3 | 5 |
|Books| 7 | 6 | 8 | 7 | 7 | 7 | 8 | 7 | 6 | 6 | 8 | 7 |

### Workforce
The production management decides how many of each furniture type they want to produce in each month, which yields the fluctuating demand for workforce.
* each worker can produce any type
* a worker needs double the time needed in the workshop to produce an item

* for each worker we can calculate with 150 workhours per month
* there are x workers employed at the moment
* there is no inventory

The following are possible strategies
1. Hiring and firing
    * at most 40 workers per month
    * 300€ extra cost for hiring
    * 420€ axtra cost for firing
2. Overtime work
    * 6 extra hours per worker
    * 5€ extra cost per hour in overtime
    
And Maybe:
3. Store surplus for future
    * costs 8€ per month per unit

Without storage:
4. Add cost of surperfluous workforce or a maximum
    * a worker with nothing to do costs 15€ extra per hour

Minimize extra costs.

## Transportation
All Produced items have a market.

* the company has two factories at two cities F1 and F2
    * possible adjustments to only have one factory site:
    * two different modes of transportation (e.g. trucks and trains)
    * alternative: One load has x space and costs y independent of amount 
* three reail centers C1,C2,C3
* monthly demand C1:4/8, C2:3/8, C3:1/8 of the total supply
    * alternative: demand for 3 month at once
* monthly supply at factories F1:1/3 of all different furniture types and F2:2/3 
* total supply == toal demand, otherwise add retail or demand for overflow
* Goal: Determine Quantity to be transported from each factory to each retail center, minimize shipping costs and satisfy demand
* cost of transportation of one item between any factory and retail center given in in 1$/item:

|    | C1 | C2 | C3 |   |
|----|----|----|----|---|
| F1 | 5  | 5  | 3  |   | 
| F2 | 6  | 4  | 1  |   | 

# Code

In [ ]:
import pyomo.environ as pyo
import numpy as np
from pyomo.opt import SolverFactory
import matplotlib.pyplot as plt
#import make_matrix as mm

## Parameters

In [ ]:
months = 12
time = range(months)

### Production

In [ ]:
factor = 10

chair_profit = [3,3,4,3,2,2,3,3,4,2,3,4]
table_profit = [4,3,5,3,5,4,4,5,4,4,3,4]
bed_profit = [4,5,4,4,3,3,4,5,4,4,3,5]
book_profit = [7,6,8,7,7,7,8,7,6,6,8,7]

avail_wood = 225 * factor
avail_metal = 114 * factor
avail_A = 120 * factor
avail_B = 50 * factor
avail_C = 150 * factor
avail_D = 100 * factor

### Workers

In [ ]:
workhours = 150
wages = 15
max_change = 10
initial_workers = 4 * factor
cost_hire = 300
cost_fire = 420
cost_overtime = 5
max_overtime = 6

## Variables

In [ ]:
def add_variables(m, mode):
    if mode == 'production' or mode =='joint':
        m.chairs = pyo.Var(time, within=pyo.NonNegativeIntegers)
        m.tables = pyo.Var(time, within=pyo.NonNegativeIntegers)
        m.bedframes = pyo.Var(time, within=pyo.NonNegativeIntegers)
        m.bookcases = pyo.Var(time, within=pyo.NonNegativeIntegers)
    if mode == 'workers' or mode == 'joint':
        m.hire = pyo.Var(time, within=pyo.NonNegativeIntegers)
        m.fire = pyo.Var(time, within=pyo.NonNegativeIntegers)
        m.overtime = pyo.Var(time, within=pyo.NonNegativeReals)
        m.workers = pyo.Var(time, within=pyo.NonNegativeReals)
        m.required_hours = pyo.Var(time, within=pyo.NonNegativeReals)
    return m


## Objective

In [ ]:
def add_objective(m,mode):
    if mode == 'production':
            m.cost = pyo.Objective(expr = sum(m.chairs[i] * chair_profit[i] 
                           + m.tables[i] * table_profit[i] 
                           + m.bedframes[i] * bed_profit[i]
                           + m.bookcases[i] * book_profit[i] for i in time), sense=pyo.maximize)
    elif mode == 'workers':
            m.cost = pyo.Objective(expr = sum(m.hire[i] * cost_hire 
                            + m.fire[i] * cost_fire
                            + m.overtime[i] * cost_overtime 
                            + (m.workers[i]*workhours + m.overtime[i] - m.required_hours[i]) * wages
                              for i in time), sense=pyo.minimize)
    elif mode == 'joint':
            m.cost = pyo.Objective(expr = sum(m.chairs[i] * chair_profit[i] 
                           + m.tables[i] * table_profit[i] 
                           + m.bedframes[i] * bed_profit[i]
                           + m.bookcases[i] * book_profit[i]
                           - m.hire[i] * cost_hire 
                           - m.fire[i] * cost_fire
                           - m.overtime[i] * cost_overtime 
                           - (m.workers[i]*workhours + m.overtime[i] - m.required_hours[i]) * wages
                              for i in time), sense=pyo.maximize)
        
    return m

## Constraints

In [ ]:
def add_constraints(m, mode):

# Add the different kinds of constraints to the m
    if mode == 'production' or mode == 'joint':
        m.materialEQ = pyo.ConstraintList()
        m.workshopEQ = pyo.ConstraintList()
    
        for i in time:
            m.materialEQ.add(1*m.chairs[i] + 2*m.tables[i] + 2*m.bedframes[i] + 3*m.bookcases[i] <= avail_wood)
            m.materialEQ.add(1*m.chairs[i] + 1*m.tables[i] + 1*m.bedframes[i] + 1*m.bookcases[i] <=avail_metal)
        
        for i in time:
            m.workshopEQ.add(2*m.chairs[i] + 3*m.tables[i] <= avail_A)
            m.workshopEQ.add(1*m.chairs[i] + 1*m.tables[i] <= avail_B)
            m.workshopEQ.add(1*m.bedframes[i] + 3*m.bookcases[i] <= avail_C)
            m.workshopEQ.add(2*m.bedframes[i] + 1*m.bookcases[i] <= avail_D)
    if mode == 'workers' or mode == 'joint':
        m.workhoursEQ = pyo.ConstraintList()
        m.workersEQ = pyo.ConstraintList()
        
        for i in time:
            m.workhoursEQ.add(m.overtime[i] + workhours*m.workers[i] >= m.required_hours[i])
            m.workhoursEQ.add(m.overtime[i] <= max_overtime * m.workers[i])
            
        m.workersEQ.add(m.workers[0] == initial_workers + m.hire[0] - m.fire[0])
        
        for i in range(0,months-1):
            m.workersEQ.add(m.workers[i+1] == m.workers[i] + m.hire[i+1] - m.fire[i+1])
            
        for i in time:
            m.workersEQ.add(m.hire[i] <= max_change)
            m.workersEQ.add(m.fire[i] <= max_change)
    if mode == 'workers':
        for i in time:
            m.workhoursEQ.add(m.required_hours[i] == 
                              2* (2*chairs[i] + 3*tables[i] + 3*bedframes[i] + 4*bookcases[i]))
    
    if mode == 'joint':
        for i in time:
            m.workhoursEQ.add(m.required_hours[i] == 
                              2* (2*m.chairs[i] + 3*m.tables[i] + 3*m.bedframes[i] + 4*m.bookcases[i]))
    
    return m

## Model

In [ ]:
def build_model(mode):

    # Step 0: Create an instance of the m
    m = pyo.ConcreteModel()
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    
    # Step 2: Define the decision varibles

    m = add_variables(m,mode)

    # Step 3: Define Objective

    m = add_objective(m, mode)             

    # Step 4: Constraints
    m = add_constraints(m, mode) 
    
    results = SolverFactory('glpk').solve(m, tee = True)
    print(results)
    # use just m.pprint() to print all variables
    m.pprint()    
    

    return m

In [ ]:
prod_m = build_model('production')

In [ ]:
chairs = [pyo.value(prod_m.chairs[i]) for i in time]
tables = [pyo.value(prod_m.tables[i]) for i in time]
bedframes = [pyo.value(prod_m.bedframes[i]) for i in time]
bookcases = [pyo.value(prod_m.bookcases[i]) for i in time]


In [ ]:
work_m = build_model('workers')

In [ ]:
joint_m = build_model('joint')